In [1]:
import argparse
import torch
import torch.backends.cudnn as cudnn
from torchvision import models
from SimCLR.data_aug.contrastive_learning_dataset import ContrastiveLearningDataset
from SimCLR.models.resnet_simclr import MyInceptionSimCLR
from SimCLR.simclr import SimCLR as SimCLR_
import multiCNN.my_inception as my_inception
from multiCNN.models import inception_3ch_to_4ch
from multiCNN.utils.general import load_backbone_state_dict

# model_names = sorted(name for name in models.__dict__
#                      if name.islower() and not name.startswith("__")
#                      and callable(models.__dict__[name]))

# parser = argparse.ArgumentParser(description='PyTorch SimCLR')
# parser.add_argument('-data', metavar='DIR', default='./datasets',
#                     help='path to dataset')
# parser.add_argument('-dataset-name', default='stl10',
#                     help='dataset name', choices=['stl10', 'cifar10'])
# parser.add_argument('-a', '--arch', metavar='ARCH', default='resnet18',
#                     choices=model_names,
#                     help='model architecture: ' +
#                          ' | '.join(model_names) +
#                          ' (default: resnet50)')
# parser.add_argument('-j', '--workers', default=12, type=int, metavar='N',
#                     help='number of data loading workers (default: 32)')
# parser.add_argument('--epochs', default=200, type=int, metavar='N',
#                     help='number of total epochs to run')
# parser.add_argument('-b', '--batch-size', default=256, type=int,
#                     metavar='N',
#                     help='mini-batch size (default: 256), this is the total '
#                          'batch size of all GPUs on the current node when '
#                          'using Data Parallel or Distributed Data Parallel')
# parser.add_argument('--lr', '--learning-rate', default=0.0003, type=float,
#                     metavar='LR', help='initial learning rate', dest='lr')
# parser.add_argument('--wd', '--weight-decay', default=1e-4, type=float,
#                     metavar='W', help='weight decay (default: 1e-4)',
#                     dest='weight_decay')
# parser.add_argument('--seed', default=None, type=int,
#                     help='seed for initializing training. ')
# parser.add_argument('--disable-cuda', action='store_true',
#                     help='Disable CUDA')
# parser.add_argument('--fp16-precision', action='store_true',
#                     help='Whether or not to use 16-bit precision GPU training.')

# parser.add_argument('--out_dim', default=128, type=int,
#                     help='feature dimension (default: 128)')
# parser.add_argument('--log-every-n-steps', default=100, type=int,
#                     help='Log every n steps')
# parser.add_argument('--temperature', default=0.07, type=float,
#                     help='softmax temperature (default: 0.07)')
# parser.add_argument('--n-views', default=2, type=int, metavar='N',
#                     help='Number of views for contrastive learning training.')
# parser.add_argument('--gpu-index', default=0, type=int, help='Gpu index.')

data="/data1/RCC/datasetSimCLR"
# dataset_name='cifar10'
dataset_name='dynamic_ct'
n_views=2
workers=12
batch_size=200
# batch_size=64
lr=0.03
out_dim=100
# out_dim=32
weight_decay=1e-4
gpu_index=1
temperature=0.07
fp16_precision=False
log_every_n_steps=100
epochs=1200
initialize_zero=True
pretrained_path=
# arch="my_inception-v3"
base_model=my_inception.inception_v3(pretrained=True,
                                     aux_logits=True,transform_input=False,attn_use=False,attn_heads=1,dropout_add=False,dropout=0.5)

inception_3ch_to_4ch(base_model,initialize_zero=initialize_zero)
if pretrained_path!=None:
    param=torch.load(pretrained_path,map_location="cuda:0")
    # model_ft.load_state_dict(param["state_dict"])
    load_backbone_state_dict(base_model, param["state_dict"])
torch.cuda.empty_cache()
# check if gpu training is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1

dataset = ContrastiveLearningDataset()

train_dataset = dataset.get_dataset(dataset_name, n_views)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=workers, pin_memory=True, drop_last=True)

model = MyInceptionSimCLR(base_model=base_model, out_dim=out_dim)

optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=weight_decay)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                        last_epoch=-1)

#  It’s a no-op if the 'gpu_index' argument is a negative integer or None.
with torch.cuda.device(gpu_index):
    simclr = SimCLR_(model=model, optimizer=optimizer, scheduler=scheduler,
                     batch_size=batch_size,n_views=n_views,temperature=temperature,
                    fp16_precision=fp16_precision,log_every_n_steps=log_every_n_steps,
                     epochs=epochs,device=device)
    simclr.train(train_loader)



  0%|          | 0/10 [00:00<?, ?it/s]

/home/shono/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:715: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
simclr.writer.log_dir

'/data1/RCC/SimCLR_logs2/2022_07_28__13_10_34'